In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
from nanomesh.volume import Volume
import numpy as np
from skimage import filters

In [ ]:
vol = Volume.load('sample_data.npy')

In [ ]:
vol.show_slice()

In [ ]:
vol.show_volume(renderer='itk')

In [ ]:
# vol.show_volume(renderer='ipyvolume')

In [ ]:
vol_gauss = vol.apply(filters.gaussian, sigma=5.0)
vol_gauss.show_slice(title='Gaussian filter')

In [ ]:
def normalize_values(image):
    return (image - image.min()) / (image.max() - image.min())

thresh = 0.5

inner = vol_gauss.apply(normalize_values)
inner = inner.apply(np.digitize, bins=[thresh])
inner.image = inner.image.astype('uint8')
inner.show_slice(title='Inner regions')

In [ ]:
inner_mesh = inner.generate_mesh_cgal(h=(1,1,1))
inner_mesh

In [ ]:
import pyvista as pv
pv.plot_itk(inner_mesh)

In [ ]:
import meshio
meshio.write('inner.vtk',inner_mesh)

In [ ]:
inner_mesh = pv.read('inner.vtk')

bounds = [100,200,100,200,100,200]
clipped = inner_mesh.clip_box(bounds)

pv.set_plot_theme("document")

p = pv.Plotter(notebook=True)

p.add_mesh(inner_mesh, style='wireframe', color='black')
p.add_mesh(clipped,  color='lightgrey')
p.show()

In [ ]:
thresh1 = 0.55
thresh2 = 0.45

two_phase = vol_gauss.apply(normalize_values)
two_phase = two_phase.apply(np.digitize, bins=[thresh1, thresh2])
two_phase.image = two_phase.image.astype('uint8')
two_phase.show_slice(title="Binary Thresholding")

In [ ]:
two_phase.show_volume(renderer='ipyvolume')

In [ ]:
two_phase_mesh = two_phase.generate_mesh_cgal()

In [ ]:
pv.plot_itk(two_phase_mesh)

In [ ]:
import meshio
meshio.write('twophase.vtk', two_phase_mesh)

In [ ]:
mesh = pv.read('twophase.vtk')

bounds = [100,200,100,200,100,200]
clipped = mesh.clip_box(bounds)
pv.set_plot_theme("document")

p = pv.Plotter(notebook=True)
p.add_mesh(mesh, style='wireframe', color='black')
p.add_mesh(clipped)
p.show(use_ipyvtk=False)